In [21]:
import os
import pickle
import asyncio
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
import gradio as gr

# Check environment variable
deployment_name = os.getenv("OPENAI_API_DEPLOYMENT_NAME")
if not deployment_name:
    raise ValueError("OPENAI_API_DEPLOYMENT_NAME environment variable is not set.")
print(f"Using deployment name: {deployment_name}")

# Initialize the model
model = AzureChatOpenAI(deployment_name=deployment_name)

# Function to read file contents
def get_file_content(filepath):
    try:
        with open(filepath, 'r') as file:
            return file.read()
    except FileNotFoundError:
        print(f"File {filepath} not found.")
        return ""

# Manage session history
store = {}
def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Function to get system prompt
def get_system_prompt():
    return get_file_content('./SystemPrompt-VSC-BusinesCentral-TestRunner.txt')

file_paths = {
    'al_endpoint': '../BusinessCentral-AL-Test-Runner-App/src/codeunit/WebApi.Codeunit.al',
    'package_json': '../VSC-BusinessCentral-Test-Runner/package.json',
    'apiclient_ts': '../VSC-BusinessCentral-Test-Runner/src/apiClient.ts',
    'project_ts': '../VSC-BusinessCentral-Test-Runner/src/project.ts',
    'testrunner_ts': '../VSC-BusinessCentral-Test-Runner/src/testRunner.ts',
    'testviewproviders_ts': '../VSC-BusinessCentral-Test-Runner/src/testViewProviders.ts',
    'extension_ts': '../VSC-BusinessCentral-Test-Runner/src/extension.ts',
}

file_contents = {key: get_file_content(path) for key, path in file_paths.items()}
systemprompt = get_system_prompt()

# Escape the curly braces
def escape_curly_braces(s):
    return s.replace('{', '{{').replace('}', '}}')

system_message = SystemMessage(content=systemprompt)
file_content_messages = [
    SystemMessage(content=f"#### This is the content of {key}: {escape_curly_braces(file_contents[key])}\n---")
    for key in file_contents
]
user_message_template = HumanMessage(content="{input}")
messages = [system_message, *file_content_messages, user_message_template]

prompt_template = ChatPromptTemplate.from_messages(messages)
print("Prompt template created.")

konnos_chat_history = get_session_history("default")
current_file_path = None
print("Chat history initialized.")

def save_chat_history_as(file_path):
    global current_file_path
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(konnos_chat_history.messages, f)
        print(f"Chat history saved as: {file_path}")
        current_file_path = file_path
        return file_path
    except Exception as e:
        print(f"Failed to save chat history: {e}")
        return None

def save_chat_history():
    global current_file_path
    if current_file_path is None:
        return None
    try:
        with open(current_file_path, 'wb') as f:
            pickle.dump(konnos_chat_history.messages, f)
        print(f"Chat history saved to: {current_file_path}")
        return current_file_path
    except Exception as e:
        print(f"Failed to save chat history: {e}")
        return None

def load_chat_history(file_path):
    global konnos_chat_history, current_file_path
    try:
        with open(file_path, 'rb') as f:
            konnos_chat_history.messages = pickle.load(f)
        current_file_path = file_path
        chat_entries = [(msg.content if isinstance(msg, HumanMessage) else "", msg.content if isinstance(msg, AIMessage) else "") for msg in konnos_chat_history.messages]
        print(f"Chat history loaded from: {file_path}")
        return chat_entries
    except Exception as e:
        print(f"Failed to load chat history: {e}")
        return []

async def converse(user_input):
    print(f"User input received: {user_input}")
    chat_history_list = konnos_chat_history.messages

    formatted_prompt = prompt_template.format(
        chat_history=chat_history_list,
        input=user_input,
        **file_contents
    )

    konnos_chat_history.add_message(HumanMessage(content=user_input))
    print("Human message added to history.")
    
    response = await asyncio.to_thread(model.invoke, formatted_prompt)
    konnos_chat_history.add_message(AIMessage(content=response.content))
    print("AI message added to history.")
    return response.content

async def interactive_conversation_stream(user_input):
    chat_history_list = konnos_chat_history.messages

    formatted_prompt = prompt_template.format(
        chat_history=chat_history_list,
        input=user_input,
        **file_contents
    )

    konnos_chat_history.add_message(HumanMessage(content=user_input))
    print("Human message added to history.")

    response = await asyncio.to_thread(model.invoke, formatted_prompt)

    konnos_chat_history.add_message(AIMessage(content=response.content))
    print("AI message added to history.")
    
    for char in response.content:
        yield char
        
def delete_message(index):
    if 0 <= index < len(konnos_chat_history.messages):
        del konnos_chat_history.messages[index]
        print(f"Deleted message at index {index}")

def delete_all_messages():
    konnos_chat_history.messages.clear()
    print("All messages deleted.")

# Gradio UI
with gr.Blocks(css=".small-button { max-width: 100px; display: inline-block !important; margin-right: 10px; }") as demo:
    chatbot = gr.Chatbot()
    message_input = gr.Textbox(placeholder="Type your message here...", label="You")

    with gr.Row():
        with gr.Column(scale=1):
            save_button = gr.Button("Save", variant="primary")
        with gr.Column(scale=1):
            save_as_button = gr.Button("Save As", variant="secondary")
        with gr.Column(scale=1):
            open_file = gr.File(label="Open Chat (.pkl)", type="filepath", file_count="single")
        with gr.Column(scale=1):
            delete_all_button = gr.Button("Delete All", variant="warning")

    async def respond(message, chat_history):
        print(f"Responding to user message: {message}")
        bot_response = "".join([chunk async for chunk in interactive_conversation_stream(message)]).strip()
        print(f"Bot response: {bot_response}")
        chat_history.append((message, bot_response))
        return chat_history, ""

    def trigger_save_as_dialog():
        return gr.File(label="Save As", type="filepath", file_count="single")

    def handle_save_as_click(file):
        if file:
            save_chat_history_as(file.name)
        return gr.update()

    def handle_save_click():
        global current_file_path
        if current_file_path is None:
            # Trigger 'Save As' dialog if no file path is set
            return trigger_save_as_dialog()
        else:
            save_chat_history()
        return gr.update()

    def handle_open_file(file):
        if file:
            chat_history = load_chat_history(file.name)
            return chat_history
        return []

    def handle_delete_all_click():
        delete_all_messages()
        return gr.update()

    save_button.click(fn=handle_save_click, inputs=[], outputs=[])
    save_as_button.click(fn=trigger_save_as_dialog, inputs=[], outputs=[save_as_button])
    #save_as_button.upload(fn=handle_save_as_click, inputs=save_as_button, outputs=[])
    open_file.upload(fn=handle_open_file, inputs=open_file, outputs=[chatbot])
    delete_all_button.click(fn=handle_delete_all_click, inputs=[], outputs=[])

    message_input.submit(respond, [message_input, chatbot], [chatbot, message_input])

demo.launch()

Using deployment name: gpt-4o
Prompt template created.
Chat history initialized.


AttributeError: 'Button' object has no attribute 'upload'

In [14]:
import os
import gradio as gr
import openai
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ChatMessageHistory
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Initialisiere LangChain und Chat-Modell
llm = AzureChatOpenAI(model="gpt-4o", api_key=os.getenv("AZURE_OPENAI_API_KEY"))
history = ChatMessageHistory()

# Lade die erste Nachricht aus der Datei
with open('../Prompts/SystemPrompt-VSC-BusinesCentral-TestRunner.txt', 'r', encoding='utf-8') as file:
    system_prompt = file.read()
history.add_message(SystemMessage(content=system_prompt))

# Lade weitere Nachrichten aus den angegebenen Dateien
file_paths = [
    '../../BusinessCentral-AL-Test-Runner-App/src/codeunit/WebApi.Codeunit.al',
    '../../VSC-BusinessCentral-Test-Runner/package.json',
    '../../VSC-BusinessCentral-Test-Runner/src/apiClient.ts',
    '../../VSC-BusinessCentral-Test-Runner/src/project.ts',
    '../../VSC-BusinessCentral-Test-Runner/src/testRunner.ts',
    '../../VSC-BusinessCentral-Test-Runner/src/testViewProviders.ts',
    '../../VSC-BusinessCentral-Test-Runner/src/extension.ts'
]

for path in file_paths:
    with open(path, 'r', encoding='utf-8') as file:
        content = file.read()
    history.add_message(HumanMessage(content=content))

# Funktion zum Speichern der Chat-Historie
def save_chat_history(history, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for message in history.messages:
            role = "system" if isinstance(message, SystemMessage) else "user" if isinstance(message, HumanMessage) else "assistant"
            file.write(f"{role}: {message.content}\n")
    print("Chat-Historie gespeichert.")

# Funktion zum Laden der Chat-Historie
def load_chat_history(filename):
    history = ChatMessageHistory()
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            role, content = line.split(": ", 1)
            if role == "user":
                history.add_message(HumanMessage(content=content.strip()))
            elif role == "assistant":
                history.add_message(AIMessage(content=content.strip()))
            elif role == "system":
                history.add_message(SystemMessage(content=content.strip()))
    print("Chat-Historie geladen.")
    return history

# Gradio Interface
def chat_interface(user_input, history):
    history.add_message(HumanMessage(content=user_input))
    response = llm(history.messages)
    response_text = response.content  # Extrahiere den Textinhalt aus der Antwort
    history.add_message(AIMessage(content=response_text))
    return [(msg.__class__.__name__.lower(), msg.content) for msg in history.messages], history

with gr.Blocks() as demo:
    gr.Markdown("## Chatbot mit LangChain, Azure Open AI und Gradio")
    chat_history = gr.State(history)
    with gr.Row():
        with gr.Column():
            chatbox = gr.Chatbot()
            user_input = gr.Textbox()
            save_button = gr.Button("Chat-Historie speichern")
            load_button = gr.Button("Chat-Historie laden")
            file_input = gr.File(label="Chat-Historie Datei", type="filepath")
            file_output = gr.File(label="Speichern unter", type="filepath")
    
    user_input.submit(chat_interface, [user_input, chat_history], [chatbox, chat_history])
    save_button.click(save_chat_history, [chat_history, file_output])
    load_button.click(load_chat_history, [file_input], chat_history)

    print("Gradio Interface gestartet.")

demo.launch()

Gradio Interface gestartet.
Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
